In [3]:
import mxnet as mx
from mxnet import init, gluon, nd, image
from mxnet.gluon.model_zoo import vision as models
import zipfile
import matplotlib.pyplot as plt

In [4]:
pretrained_net = models.resnet18_v2(pretrained=True)
print(pretrained_net)

Model file is not found. Downloading.
ResNetV2(
  (features): HybridSequential(
    (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=True, use_global_stats=False, in_channels=3)
    (1): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
    (3): Activation(relu)
    (4): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
    (5): HybridSequential(
      (0): BasicBlockV2(
        (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv1): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv2): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): BasicBlockV2(
 

In [ ]:
def load_data(route='/Users/Sinyer/Documents/Sinyer/pydata/temp'):
    
    fname = gluon.utils.download(
        'https://apache-mxnet.s3-accelerate.amazonaws.com/gluon/dataset/hotdog.zip',
        path=route, sha1_hash='fba480ffa8aa7e0febbb511d181409f899b9baa5')
    with zipfile.ZipFile(fname, 'r') as f:
        f.extractall(data_dir)
        
    train_augs = [image.HorizontalFlipAug(.5),image.RandomCropAug((224,224))]
    test_augs = [image.CenterCropAug((224,224))]

    train_imgs = gluon.data.vision.ImageFolderDataset(
        data_dir+'/hotdog/train', transform=lambda X, y: transform(X, y, train_augs))
    test_imgs = gluon.data.vision.ImageFolderDataset(
        data_dir+'/hotdog/test', transform=lambda X, y: transform(X, y, test_augs))
    return train_imgs, test_imgs

def transform(data, label, augs):
    data = data.astype('float32')
    for aug in augs:
        data = aug(data)
    data = nd.transpose(data, (2,0,1))
    return data, nd.array([label]).asscalar().astype('float32')

In [ ]:
train_imgs, test_imgs = load_data()
pretrained_net = models.resnet18_v2(pretrained=True)

net = models.resnet18_v2(classes=2)
net.features = pretrained_net.features
net.output.initialize(init.Xavier())

In [ ]:
epochs=10
batch_size=64

train_data = gluon.data.DataLoader(train_imgs, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(test_imgs, batch_size)

net.collect_params().reset_ctx(ctx)
net.hybridize()
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.01, 'wd': 1e-3})

utils.train(train_data, test_data, net, loss, trainer, ctx, epochs)

In [ ]:
def classify_hotdog(net, fname):
    with open(fname, 'rb') as f:
        img = image.imdecode(f.read())
    data, _ = transform(img, -1, test_augs)
    plt.imshow(data.transpose((1,2,0)).asnumpy()/255)
    
    data = data.expand_dims(axis=0)
    out = net(data.as_in_context(ctx[0]))
    out = nd.SoftmaxActivation(out)
    pred = int(nd.argmax(out, axis=1).asscalar())
    prob = out[0][pred].asscalar()
    label = train_imgs.synsets
    return 'prob=%f, %s'%(prob, label[pred])

In [ ]:
classify_hotdog(net, '../img/real_hotdog.jpg')